# 1. 컨텐츠 기반 추천

## 1-1. 컨텐츠 기반 모델

1) 정의 : 사용자가 이전에 구매한 상품 중에서 좋아하는 상ㅍ무들과 유사한 상품들을 추천하는 방법

2) Represented Items : 텍스트나 이미지 같은 아이템을 벡터 형태로 표현하여, 벡터들간의 유사도를 계산해 벡터1부터 N까지 자신과 유사한 벡터를 추출한다.

3) 유사도 함수
- 유클리디안 유사도 : 문서간의 유사도를 계산(1 / (유클리디안 거리 + 1e-05))
ㄴ 장점 : 계산하기 쉽다
ㄴ 단점 : p와 q의 분포가 다르거나 범위가 다른 경우에 상관성을 놓친다

- 코사인 유사도 : 문서간의 유사도를 계산
ㄴ 장점 : 벡터의 크기가 중요하지 않은 경우에 거리를 측정하기 위한 메트릭으로 사용
ㄴ 단점 : 벡터의 크기가 중요한 경우에 대해 잘 작동하지 않는다

- 피어슨 유사도 : 상관관계를 분석할 때 주로 사용한다

- 자카드 유사도 : 집합에서 얼마만큼의 결합된 부분이 있는지를 확인

- 그외 여러 유사도 함수가 있으므로 scikit-learn 등을 확인하여 여러 유사도 함수를 확인해 볼 필요가 있다
- 여러 유사도 함수를 확인한 뒤 유사도 함수 별로 가중치를 주는 등 결합하여 사용할 수 있다, 또한 각각의 매트릭스 별로 비교할 수도 있다






## 1-2. TF-IDF

1) TF-IDF : 텍스트 아이템을 벡터로 표현하는 방법
- 정의

ㄴ 단어빈도(TF) : 특정 문서 d에 특정 단어 t가 얼마나 자주 등장하는지(등장횟수)

ㄴ 역문서빈도(DF) : 전체 문서에서 특정 단어 t가 얼마나 자주 등장하는지(t가 등장한 문서의 수)

ㄴ IDF(d,t) : DF에 반비례하는 수 IDF(d, t) = log(n / 1 + df(t))

ㄴ TF와 DF를 통해 "다른 문서에서는 등장하지 않지만 특정 문서에서만 자주 등장하는 단어"를 찾아 문서 내 단어의 가중치를 계산


- 용도 : 문서의 핵심어를 추출, 문서들 사이의 유사도 계산, 검색 결과의 중요도를 정하는 작업 등에 사용

- TF(d, t) * IDF(d, t) = TF-IDF(d, t)

- 사용하는 이유?

ㄴ item이라는 컨텐츠를 벡터로 "Feature Extract" 과정을 수행

ㄴ 빈도수를 기반으로 많이 나오는 중요한 단어들을 잡아준다(counter vectorizer)

ㄴ 하지만 이는 단순 빈도만을 계산하기에 조사/관사처럼 의미없지만 문장에 많이 등장하는 단어들을 높게 칠 수 있음. 이러한 단어들에 패널티를 주어 중요한 단어를 잡아내는 기법이 TF-IDF


- 장점 : 직관적인 해석

- 단점 : 대규모 말뭉치를 다룰 때 메모리상의 문제가 발생하고, 높은 차원을 가지게 된다. 매우 sparse한 형태의 데이터..


In [1]:
docs = ['먹고 싶은 사과', # 문서1
       '먹고 싶은 바나나',# 문서2
       '길고 노란 바나나 바나나',# 문서3
       '저는 과일이 좋아요']# 문서4

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [3]:
#문장을 counter vectorizer 형태로 표현

countvect = vect.fit_transform(docs)
countvect

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [4]:
countvect.toarray()

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]], dtype=int64)

In [5]:
# 딕셔너리 형태로 각각의 단어의 컬럼 위치를 보여준다.
vect.vocabulary_

{'먹고': 3,
 '싶은': 6,
 '사과': 5,
 '바나나': 4,
 '길고': 1,
 '노란': 2,
 '저는': 7,
 '과일이': 0,
 '좋아요': 8}

In [6]:
sorted(vect.vocabulary_)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [7]:
import pandas as pd

countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1','문서2','문서3','문서4']
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


In [8]:
# 0번 문서는 1번 문서와 유사하다는 결론을 얻을 수 있다. 따라서 0번 문서를 본 사용자에게 1번 문서를 추천할 수 있다.
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(countvect_df, countvect_df)

array([[1.        , 0.66666667, 0.        , 0.        ],
       [0.66666667, 1.        , 0.47140452, 0.        ],
       [0.        , 0.47140452, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [9]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfvect = vect.fit(docs)

In [11]:
tfvect_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfvect_df.index = ['문서1','문서2','문서3','문서4']
tfvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
문서2,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
문서3,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
문서4,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfvect_df, tfvect_df)

array([[1.        , 0.60784064, 0.        , 0.        ],
       [0.60784064, 1.        , 0.42980824, 0.        ],
       [0.        , 0.42980824, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])